In [1]:
# project
import os
import sys 
project_dir = os.path.join(os.path.expanduser('~'), 'git', 'diffsim-sysid')
sys.path.insert(1, project_dir)
# math
import numpy as np
import torch
import pytorch3d
import pytorch3d.transforms
# physics
import warp as wp
import warp.sim as wps
# plots
import matplotlib as mpl
from matplotlib import pyplot as plt
from src.plot import set_fig_size, set_mpl_format
# config
from src.config import cfg
# dataset
from src.dataset import load_train_data
from src.data.visualize import print_example

# setup
set_mpl_format()
wp.init()

2024-04-03 18:58:51.866610: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Warp 1.0.2 initialized:
   CUDA Toolkit 11.5, Driver 11.4
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "NVIDIA GeForce GTX 1050 Ti with Max-Q Design" (4 GiB, sm_61, mempool enabled)
   Kernel cache:
     /home/jonathan/.cache/warp/1.0.2


### Config

In [2]:
# fill in your directory set up here
config_fp = os.path.join(project_dir, f'cfg/templates/comp-movi_a.yaml')
cfg.merge_from_file(config_fp)
cfg.DATA.path = os.path.join(project_dir, 'data/sets/')
print(cfg)

DATA:
  fps: 12
  fpv: 24
  height: 256
  max_objs: 10
  path: /home/jonathan/git/diffsim-sysid/data/sets/
  set: movi_a
  width: 256
EVAL:
  OUTPUT:
    FN:
      config: config.yaml
      log: log.txt
      pred: pred.csv
    path: 
  PARAM:
    DENSITY:
      include: True
      prior: 1.35
    FRICTION:
      include: True
      prior: 0.5
    RESTITUTION:
      include: True
      prior: 0.5
SYS_ID:
  GEOM:
    include: True
    spm: 100
  OPTIM:
    beta1: 0.9
    beta2: 0.999
    decay: 0.0
    lr: 0.01
    optim: adam
  VIS:
    include: True
    spp: 4
    weight: 1.0
  iter: 4
  rand: False


### Data

In [3]:
train_ds, ds_info = load_train_data(cfg)
example = next(iter(train_ds))

2024-04-03 18:58:54.988100: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-03 18:58:55.001489: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-03 18:58:55.001936: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-03 18:58:55.003622: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operation

In [4]:
vid_id = example['metadata']['video_name']
obj_count = example['metadata']['num_instances']
print(f'Video: {vid_id}, Num objects: {obj_count}')

Video: b'1680', Num objects: 8


In [6]:
example['instances']['velocities'].shape

(8, 24, 3)

In [7]:
example['instances'].keys()

dict_keys(['angular_velocities', 'bbox_frames', 'bboxes', 'bboxes_3d', 'color', 'color_label', 'friction', 'image_positions', 'mass', 'material_label', 'positions', 'quaternions', 'restitution', 'shape_label', 'size_label', 'velocities', 'visibility'])

### Scene

In [5]:
# simulation parameters
sim_duration = 1.0
# control frequency
fps = 12
frame_dt = 1.0 / float(fps)
frame_steps = int(sim_duration / frame_dt)
# sim frequency
sim_substeps = 8 * 5
sim_steps = frame_steps * sim_substeps
sim_dt = frame_dt / sim_substeps

### Physics

In [6]:
CLEVR_SHAPE_NAMES = ("cube", "cylinder", "sphere")
CLEVR_SIZE_NAMES = ('small', 'large')
CLEVR_SIZES = (0.7, 1.4)
CLEVR_DENSITIES = (1.1, 2.7)

phys_device = wp.get_cuda_device()
phys_builder = wps.ModelBuilder(up_vector=(0, 0, 1))

ground_mu = 0.3
ground_restitution = 0.5

# ground
phys_builder.set_ground_plane(mu=ground_mu, 
                              restitution=ground_restitution)

for obj_idx in range(obj_count):
    obj_class_idx = example['instances']['shape_label'][obj_idx]
    obj_class = CLEVR_SHAPE_NAMES[obj_class_idx]
    
    obj_scale_idx = example['instances']['size_label'][obj_idx]
    obj_scale = CLEVR_SIZES[obj_scale_idx] / 2.0
    obj_pos = example['instances']['positions'][obj_idx][0]
    obj_ori = example['instances']['quaternions'][obj_idx][0]

    obj_vel = example['instances']['velocities'][obj_idx][0]

    obj_material = example['instances']['material_label'][obj_idx]
    obj_density = CLEVR_DENSITIES[obj_material]

    obj_mass = float(example['instances']['mass'][obj_idx])
    obj_fric = example['instances']['friction'][obj_idx]
    obj_restitution = example['instances']['restitution'][obj_idx]

    phys_builder.add_particle(
        pos=obj_pos,
        vel=obj_vel,
        radius=obj_scale,
        mass=obj_mass,
        
    )

phys_model = phys_builder.finalize(phys_device, requires_grad=False)

phys_states = [phys_model.state(requires_grad=False) for _ in range(sim_steps + 1)]

Warp DeprecationWarning: Support for built-in functions called with non-Warp array types, such as lists, tuples, NumPy arrays, and others, will be dropped in the future. Use a Warp type such as `wp.vec`, `wp.mat`, `wp.quat`, or `wp.transform`.
Module warp.sim.collide load on device 'cuda:0' took 99.37 ms


In [8]:
def capture_phys(device, model, integrator, states):
    # tape tracks computation graph
    tape = wp.Tape()
    wp.capture_begin(device=device)
    with tape:
        for i in range(sim_steps):
            states[i].clear_forces()
            wp.sim.collide(model, states[i])
            integrator.simulate(model, states[i], states[i + 1], sim_dt)
    graph = wp.capture_end()

    return graph, tape


def get_trajectory(states):
    trajectory = []
    for state in states[::sim_substeps]:
        trajectory.append(state.particle_q.numpy().squeeze())
    return np.array(trajectory)


def distance(target_pose, pred_pose):
    return torch.linalg.norm(target_pose - pred_pose)

In [9]:
# initial run 
phys_integrator = wp.sim.SemiImplicitIntegrator()
phys_graph, phys_tape = capture_phys(phys_device, phys_model, phys_integrator, phys_states)
wp.capture_launch(phys_graph)

Module warp.utils load on device 'cuda:0' took 15.55 ms
Module warp.sim.inertia load on device 'cuda:0' took 10.87 ms
Module warp.sim.model load on device 'cuda:0' took 0.19 ms
Module warp.sim.integrator load on device 'cuda:0' took 7.98 ms
Module warp.sim.particles load on device 'cuda:0' took 2.75 ms
Module warp.sim.utils load on device 'cuda:0' took 14.29 ms
Module warp.sim.integrator_euler load on device 'cuda:0' took 222.88 ms
Module warp.sim.articulation load on device 'cuda:0' took 38.44 ms
Module warp.sim.integrator_featherstone load on device 'cuda:0' took 76.27 ms
Module warp.sim.integrator_xpbd load on device 'cuda:0' took 148.89 ms


In [21]:
positions = np.zeros_like(example['instances']['positions'])
quaternions = np.zeros_like(example['instances']['quaternions'])

for i in range(len(phys_states)):
    pos_ori = phys_states[i].body_q.numpy()
    positions[i] = pos_ori[i, :3]
    quaternions[i] = pos_ori[i, 3:]

(8, 24, 3)
(8, 24, 4)
(8, 7)
(8, 7)
(8, 7)
(8, 7)
(8, 7)
(8, 7)
(8, 7)
(8, 7)
(8, 7)


IndexError: index 8 is out of bounds for axis 0 with size 8

In [23]:
phys_states[0].body_q.numpy()[:, :3]

(6, 7)

In [19]:
example['instances']['positions'].shape

(8, 24, 3)

In [20]:
example['instances']['quaternions'].shape

(8, 24, 4)

In [ ]:
phys_states[0].body_q.numpy()[0][:3]

In [ ]:
phys_states[-1].body_q.numpy()[:, 2]